# Taller LU
Nombre: Rommel Rivera

In [7]:
%load_ext autoreload

In [8]:
# ----------------------------- logging --------------------------
import logging
from sys import stdout
from datetime import datetime

logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(levelname)s] %(message)s",
    stream=stdout,
    datefmt="%m-%d %H:%M:%S",
)
logging.info(datetime.now())

[01-13 21:03:27][INFO] 2026-01-13 21:03:27.941737


In [9]:
%autoreload 2
from src import eliminacion_gaussiana


[01-13 21:03:30][INFO] 2026-01-13 21:03:30.607175


In [10]:
A = [[1, 3, 4], [2, 1, 3], [4, 2, 1]]

In [12]:
import numpy as np


# ####################################################################
def gauss_jordan(A: np.ndarray) -> np.ndarray:
    """Realiza la eliminación de Gauss-Jordan

    ## Parameters

    ``A``: matriz del sistema de ecuaciones lineales. Debe ser de tamaño n-by-(k), donde n es el número de incógnitas.

    ## Return

    ``A``: matriz reducida por filas.

    """
    if not isinstance(A, np.ndarray):
        logging.debug("Convirtiendo A a numpy array.")
        A = np.array(A, dtype=float)
    n = A.shape[0]

    for i in range(0, n):  # loop por columna

        # --- encontrar pivote
        p = None  # default, first element
        for pi in range(i, n):
            if A[pi, i] == 0:
                # must be nonzero
                continue

            if p is None:
                # first nonzero element
                p = pi
                continue

            if abs(A[pi, i]) < abs(A[p, i]):
                p = pi

        if p is None:
            # no pivot found.
            raise ValueError("No existe solución única.")

        if p != i:
            # swap rows
            logging.debug(f"Intercambiando filas {i} y {p}")
            _aux = A[i, :].copy()
            A[i, :] = A[p, :].copy()
            A[p, :] = _aux

        # --- Eliminación: loop por fila
        # for j in range(i + 1, n): # Eliminación gaussiana
        for j in range(n):  # Gauss-Jordan
            if j == i:
                continue  # skip pivot row

            m = A[j, i] / A[i, i]
            A[j, i:] = A[j, i:] - m * A[i, i:]

            # dividir para la diagonal
        A[i, :] = A[i, :] / A[i, i]

        logging.info(f"\n{A}")

    if A[n - 1, n - 1] == 0:
        raise ValueError("No existe solución única.")

        print(f"\n{A}")
    # # --- Sustitución hacia atrás
    # solucion = np.zeros(n)
    # solucion[n - 1] = A[n - 1, n] / A[n - 1, n - 1]

    # for i in range(n - 2, -1, -1):
    #     suma = 0
    #     for j in range(i + 1, n):
    #         suma += A[i, j] * solucion[j]
    #     solucion[i] = (A[i, n] - suma) / A[i, i]

    return A

In [13]:
from pprint import pprint

pprint(A)
gauss_jordan(A)

[[1, 3, 4], [2, 1, 3], [4, 2, 1]]
[01-13 21:03:41][INFO] 
[[  1.   3.   4.]
 [  0.  -5.  -5.]
 [  0. -10. -15.]]
[01-13 21:03:41][INFO] 
[[ 1.  0.  1.]
 [-0.  1.  1.]
 [ 0.  0. -5.]]
[01-13 21:03:41][INFO] 
[[ 1.  0.  0.]
 [-0.  1.  0.]
 [-0. -0.  1.]]


array([[ 1.,  0.,  0.],
       [-0.,  1.,  0.],
       [-0., -0.,  1.]])

In [14]:
# Poner matriz identidad a la derecha
n = len(A)
A_aug = np.hstack((A, np.eye(n)))
A_aug

array([[1., 3., 4., 1., 0., 0.],
       [2., 1., 3., 0., 1., 0.],
       [4., 2., 1., 0., 0., 1.]])

In [15]:
np.vstack((A, np.eye(n)))

array([[1., 3., 4.],
       [2., 1., 3.],
       [4., 2., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [16]:
_m_inv = gauss_jordan(A_aug)
_m_inv

[01-13 21:03:50][INFO] 
[[  1.   3.   4.   1.   0.   0.]
 [  0.  -5.  -5.  -2.   1.   0.]
 [  0. -10. -15.  -4.   0.   1.]]
[01-13 21:03:50][INFO] 
[[ 1.   0.   1.  -0.2  0.6  0. ]
 [-0.   1.   1.   0.4 -0.2 -0. ]
 [ 0.   0.  -5.   0.  -2.   1. ]]
[01-13 21:03:50][INFO] 
[[ 1.   0.   0.  -0.2  0.2  0.2]
 [-0.   1.   0.   0.4 -0.6  0.2]
 [-0.  -0.   1.  -0.   0.4 -0.2]]


array([[ 1. ,  0. ,  0. , -0.2,  0.2,  0.2],
       [-0. ,  1. ,  0. ,  0.4, -0.6,  0.2],
       [-0. , -0. ,  1. , -0. ,  0.4, -0.2]])

In [14]:
_m_inv[:, n:]

array([[-0.2,  0.2,  0.2],
       [ 0.4, -0.6,  0.2],
       [-0. ,  0.4, -0.2]])

# Comprobando respuesta

In [8]:
np.linalg.inv(np.array(A))

array([[-0.2,  0.2,  0.2],
       [ 0.4, -0.6,  0.2],
       [ 0. ,  0.4, -0.2]])

In [9]:
import numpy as np

# --- 1. MÉTODOS MATEMÁTICOS (Núcleo) ---

def descomposicion_LU(A: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """Realiza la descomposición LU. Se ejecuta UNA vez por matriz."""
    A = np.array(A, dtype=float).copy() 
    n = A.shape[0]
    L = np.zeros((n, n), dtype=float)

    for i in range(0, n):
        if A[i, i] == 0:
            raise ValueError(f"Pivote 0 encontrado en fila {i}.")
        
        L[i, i] = 1
        for j in range(i + 1, n):
            m = A[j, i] / A[i, i]
            A[j, i:] = A[j, i:] - m * A[i, i:]
            L[j, i] = m
            
    return L, A # Retornamos L y U (A mutada)

def sustitucion_adelante(L, b):
    """Resuelve Ly = b"""
    n = len(b)
    y = np.zeros(n)
    for i in range(n):
        suma = sum(L[i, j] * y[j] for j in range(i))
        y[i] = (b[i] - suma) / L[i, i] 
    return y

def sustitucion_atras(U, y):
    """Resuelve Ux = y"""
    n = len(y)
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        suma = sum(U[i, j] * x[j] for j in range(i + 1, n))
        x[i] = (y[i] - suma) / U[i, i]
    return x

# --- 2. LÓGICA DE OPTIMIZACIÓN ---

def resolver_multiples_b(nombre_matriz, A_input, lista_vectores_b):
    """
    Toma una matriz y una lista de vectores.
    Optimización: Calcula LU solo 1 vez y resuelve para N vectores.
    """
    A = np.array(A_input)
    n = A.shape[0]
    
    print(f"\n{'='*60}")
    print(f"Procesando Matriz {nombre_matriz} (Tamaño {n}x{n})")
    
    # [OPTIMIZACIÓN] Paso costoso (LU) fuera del bucle de vectores
    try:
        L, U = descomposicion_LU(A)
        # print(f"  -> Descomposición LU exitosa.") 
    except ValueError as e:
        print(f"  -> Error descomponiendo {nombre_matriz}: {e}")
        return

    # Paso rápido (Sustituciones) repetido para cada b compatible
    count = 0
    for i, b_vals in enumerate(lista_vectores_b):
        b = np.array(b_vals, dtype=float)
        
        # Filtrar: Solo procesamos vectores del tamaño correcto
        if len(b) != n:
            continue 
            
        # Proceso ligero O(n^2)
        y = sustitucion_adelante(L, b)
        x = sustitucion_atras(U, y)
        
        count += 1
        print(f"  Solución para B{i+1} {b_vals}: \n    x = {np.round(x, 4)}")

    if count == 0:
        print("  -> Ningún vector B compatible encontrado para esta matriz.")

# --- 3. DATOS DE ENTRADA ---

# Matrices
A1 = [[1, 3, 4], [2, 1, 3], [4, 2, 1]]
A2 = [[1, 2, 3], [0, 1, 4], [5, 6, 1]]
A3 = [[4, 2, 1], [2, 1, 3], [1, 3, 4]]
A4 = [[2, 4, 6, 1], [4, 7, 5, -6], [2, 5, 18, 10],[6, 12, 38, 16]]

# Vectores (Los ponemos todos en una lista común para que el programa filtre)
todos_los_b = [
    [1, 2, 4],       # B1
    [3, -5, 2],      # B2
    [7, 8, -1],      # B3
    [1, 2, 4, 5],    # B4
    [3, -5, 2, 6],   # B5
    [7, 8, -1, 0]    # B6
]

# --- 4. EJECUCIÓN ---

matrices = [("A1", A1), ("A2", A2), ("A3", A3), ("A4", A4)]

for nombre, mat in matrices:
    resolver_multiples_b(nombre, mat, todos_los_b)


Procesando Matriz A1 (Tamaño 3x3)
  Solución para B1 [1, 2, 4]: 
    x = [ 1. -0. -0.]
  Solución para B2 [3, -5, 2]: 
    x = [-1.2  4.6 -2.4]
  Solución para B3 [7, 8, -1]: 
    x = [ 0.  -2.2  3.4]

Procesando Matriz A2 (Tamaño 3x3)
  Solución para B1 [1, 2, 4]: 
    x = [ 14.5 -12.    3.5]
  Solución para B2 [3, -5, 2]: 
    x = [-69.5  61.  -16.5]
  Solución para B3 [7, 8, -1]: 
    x = [-19.  16.  -2.]

Procesando Matriz A3 (Tamaño 3x3)
  -> Error descomponiendo A3: Pivote 0 encontrado en fila 1.

Procesando Matriz A4 (Tamaño 4x4)
  Solución para B4 [1, 2, 4, 5]: 
    x = [-7.6778  3.1333  0.8222 -1.1111]
  Solución para B5 [3, -5, 2, 6]: 
    x = [19.6 -5.2 -3.4  5. ]
  Solución para B6 [7, 8, -1, 0]: 
    x = [12.4222 -0.0667 -3.5778  3.8889]


Link: https://github.com/RommelRam/Metodos-Numericos.git